In [60]:
import pandas as pd
import chardet

caminho_arquivo = "/home/juniorsilvadev/engenharia-dados/engenharia-dados-com-python-e-banco-de-dados/PostgreSQL/Arquivos JSON/V_OCORRENCIA_AMPLA.json"

with open(caminho_arquivo, "rb") as f:
    raw_data = f.read(10000)
    result = chardet.detect(raw_data)
    encoding_detectado = result['encoding']

df = pd.read_json(caminho_arquivo, encoding=encoding_detectado)

In [61]:
df = pd.read_json(caminho_arquivo, encoding=encoding_detectado)

In [62]:
colunas = ['Numero_da_Ocorrencia', 'Classificacao_da_Ocorrência', 'Data_da_Ocorrencia', 'Municipio', 'UF', 'Regiao', 'Nome_do_Fabricante']
df = df[colunas]

In [63]:
df.rename(columns={'Classificacao_da_Ocorrência': 'Classificacao_da_Ocorrencia'}, inplace=True)
df.columns = df.columns.str.strip().str.lower() # Remover espaços extras e converter os nomes das colunas para minúscula

In [64]:
print(df.columns.tolist())
df.head()

['numero_da_ocorrencia', 'classificacao_da_ocorrencia', 'data_da_ocorrencia', 'municipio', 'uf', 'regiao', 'nome_do_fabricante']


,numero_da_ocorrencia,classificacao_da_ocorrencia,data_da_ocorrencia,municipio,uf,regiao,nome_do_fabricante
0,7762,Incidente,2018-03-21,SÃO PAULO,SP,Sudeste,AGUSTA
1,7759,Acidente,2018-03-14,MONTES CLAROS,MG,Sudeste,CESSNA AIRCRAFT
2,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT
3,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT
4,7757,Incidente Grave,2018-03-18,TORRES,RS,Sul,PIPER AIRCRAFT


In [65]:
df['data_da_ocorrencia'] = pd.to_datetime(df['data_da_ocorrencia'])

In [66]:
from datetime import datetime

ano_atual = datetime.now().year
print(ano_atual)

2025


In [67]:
df = df[df['data_da_ocorrencia'].dt.year == 2024]

Conexão avançada com create engine

In [68]:
# pip install sqlalchemy
# Tipos de dados: Integer, String, Date, VARCHAR
# Conexão com PostgreSQL, MySQL, SQLite e muito mais com auto desemprenho

from sqlalchemy import create_engine, Integer, String, Date, VARCHAR, text

dbname   = 'anac_db'
user     = 'postgres'
password = 'root'
host     = 'localhost'
port     = '5432'

conexao_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

try:
    engine = create_engine(conexao_str)
    print("Conexão bem-sucedida ao PostgreSQL!")
except Exception as e:
    print('Error ao conectar com banco de dados:', e)

Conexão bem-sucedida ao PostgreSQL!


In [69]:
nome_tabela = 'anac_sqlalchemy'
ano_2024 = 2024

# Estabelece a conexão
cursor = engine.connect()

# Cria a consulta SQL como texto executável
delete = text(f'DELETE FROM public.{nome_tabela} WHERE EXTRACT(YEAR FROM "data_da_ocorrencia"::DATE) = {ano_atual}')

# Executa a consulta
cursor.execute(delete)

# Agora insere os dados do DataFrame na tabela
df.to_sql(nome_tabela, engine, index=False, if_exists='append', dtype={
    'Numero_da_Ocorrencia': Integer,
    'Classificacao_da_Ocorrencia': VARCHAR(50),
    'Data_da_Ocorrencia': Date,
    'Municipio': VARCHAR(50),
    'UF': VARCHAR(30),
    'Regiao': VARCHAR(50),
    'Nome_do_Fabricante': VARCHAR(50)
})

# replace - sobrescreve toda a tabela
# append -  adiciona dados ao fim da tabela

engine.dispose()
cursor.close()